# **1. Cargar dataset de imágenes en memoria utilizando openCV**

In [2]:
paths=["/kaggle/input/brain-tumor-mri-dataset/Training/",
    "/kaggle/input/brain-tumor-mri-dataset/Testing/"]

In [3]:
import os 
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def imshow(img):
    fig,ax = plt.subplots(1,1,figsize=(8,8))
    ax.imshow(img,cmap='gray')

In [5]:
possible_labels=os.listdir(paths[0])

In [6]:
possible_labels

In [7]:
data=[]

for i,path in enumerate(paths):
    for label_int, label_string in enumerate(possible_labels):
        for filename in glob.glob(path+label_string+'/*.jpg'):
            img=cv2.imread(filename)
            data.append( [label_int,cv2.resize(img,(320,320))] )
len(data)

In [8]:
import random
random.Random(0).shuffle(data) 

x_train=[]
y_train=[]

x_val=[]
y_val=[]

x_test=[]
y_test=[]

for i, sample in enumerate(data):
    label=sample[0]
    img=sample[1]
    if i<= 0.8*len(data):
        x_train.append(img)
        y_train.append(label)
    elif i>0.8*len(data) and i<=0.9*len(data):
        x_val.append(img)
        y_val.append(label)
    else:
        x_test.append(img)
        y_test.append(label)

x_train=np.array(x_train)
x_val=np.array(x_val)
x_test=np.array(x_test)

y_train=np.array(y_train)
y_val=np.array(y_val)
y_test=np.array(y_test)

In [9]:
x_test.shape

In [10]:
imshow(x_train[0])

In [11]:
y_train[0]

In [12]:
possible_labels[y_train[0]]

# **2. Definición y entrenamiento de la red neuronal convolucional**

In [13]:
import tensorflow as tf
y_trainOneHot=tf.one_hot(y_train,len(possible_labels))
y_valOneHot=tf.one_hot(y_val,len(possible_labels))
y_testOneHot=tf.one_hot(y_test,len(possible_labels))


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

In [15]:
def cnn_model():
    model = Sequential()
    model.add(Convolution2D(32,(3,3),input_shape=(320,320,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(30,activation='relu'))
    model.add(Dense(4,activation='softmax'))
    model.compile(optimizer='SGD', loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [16]:
model=cnn_model()
model.summary()

In [17]:
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
checkpoint=ModelCheckpoint('checkpoint/model.{epoch:d}.h5',save_best_only=False, save_freq='epoch')
tensorboard_callback=TensorBoard('logs/cnn_logs',histogram_freq=1)

In [18]:
x_train.shape

In [19]:
x_val.shape

In [20]:
model.fit(x_train,
          y_trainOneHot,
          epochs=20,
          batch_size=100,
          validation_data=(x_val,y_valOneHot),
          callbacks=[tensorboard_callback,checkpoint]
)

# **3. Predicción y Evaluación del modelo sobre el dataset de prueba**

In [25]:
possible_labels[np.argmax(model.predict(x_test[0:1]))]

In [26]:
possible_labels[y_test[0]]

In [27]:
model.evaluate(x=x_test,y=y_testOneHot)

# **4. Tensorboard para escoger el mejor modelo**

In [28]:
# Download Ngrok to tunnel the tensorboard port to an external port
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

# Run tensorboard as well as Ngrox (for tunneling as non-blocking processes)
import os
import multiprocessing


pool = multiprocessing.Pool(processes = 10)
results_of_processes = [pool.apply_async(os.system, args=(cmd, ), callback = None )
                        for cmd in [
                        f"tensorboard --logdir ./logs/ --host 0.0.0.0 --port 6006 &",
                        "./ngrok authtoken 285s9wc55xTlRKZc2ryBmv0jhpx_5W3k4zwJrPTXA1CLTEGY3 &",
                        "./ngrok http 6006 &"
                        ]]

In [29]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [30]:
model.load_weights("./checkpoint/model.18.h5")

In [31]:
model.evaluate(x=x_test,y=y_testOneHot)